# Example 16: KES Block thresholds with PSA electrode

This script evaluates the KES block threshold of myelinated fibers when stimulated with a PSA electrode. It is a close replication of results published by Bhadra et al (2006).

In [2]:
import sys
sys.path.append("../")
import nrv
import matplotlib.pyplot as plt
import numpy as np

#nrv.parameters.set_nrv_verbosity(2)

t_sim = 20                  		#Simulation Duration [ms]

# axon def
y_a = 0								# axon y position, in [um]
z_a = 0								# axon z position, in [um]
N_node = 51                 		# 21 nodes of Ranvier   -- was 51 in orignal paper
model = 'MRG'						#Myelinated Axon model
material =  'endoneurium_bhadra'    #material for the endoneurium

# test pulse
t_start = t_sim-5   						# Test pulse start, in [ms]
t_duration = 0.1						# Test pulse duration, in [ms]
t_amplitude = 5						# Test pulse amplitude, in [nA]

#Axon ranges from 2µm to 20µm
d_min = 2
d_max = 20
n_diam = 10
d_axon_list = np.round(np.linspace(d_min,d_max,num=n_diam))


d_elect = 100				#Electrode Distance list, in [um]
max_amp = 100		        #Max KES amplitude for the Binary search

KES_freq = 10                           #KES block frequency list, in [kHz]			
amp_tol = 5								#Amplitude tolerance for the search, in %

dt = 0.005

stim = nrv.stimulus()                     #dummy stim
stim_extra = nrv.stimulation(material)      #analytical stimulation, no FEM

for diam in d_axon_list:
    print(f"Axon diameter: {diam}µm")
    threshold_list= []
    L = nrv.get_length_from_nodes(diam,N_node)	#Return L of myelinated axon for a given number of NoR, in [um]

    axon1 = nrv.myelinated(y=y_a,z=z_a,d=diam, L=L, rec="nodes",model=model)
    n_node = len(axon1.x_nodes)
    x_elec = axon1.x_nodes[n_node//2]
    elec_1 = nrv.point_source_electrode(x_elec, d_elect, 0)
    stim_extra.add_electrode(elec_1, stim)        
    axon1.attach_extracellular_stimulation(stim_extra)

    threshold = nrv.blocking_threshold_from_axon(axon1,block_freq = KES_freq, amp_max  =max_amp,
                                             amp_tol = amp_tol,t_sim= t_sim,t_start=t_start,
                                             t_duration = t_duration, t_amplitude = t_amplitude,
                                             b_start = 1)
    
    print ("Block Threshold is: " + str(int(threshold)) +"uA.")
    threshold_list.append(threshold)
    del axon1


Axon diameter: 2.0µm
NRV INFO: Iteration number 1, testing block current amplitude 600 uA
NRV WARNING: no loaded footprints: computation will be done anyway
NRV INFO: ... Iteration simulation performed in 9.084656238555908 s
NRV INFO: ... Spike blocked
NRV INFO: Iteration number 2, testing block current amplitude 0 uA
NRV INFO: ... Iteration simulation performed in 8.743825197219849 s
NRV INFO: ... Spike not blocked
NRV INFO: Iteration number 3, testing block current amplitude 300.0 uA
NRV INFO: ... Iteration simulation performed in 8.280965089797974 s
NRV INFO: ... Spike blocked
NRV INFO: Iteration number 4, testing block current amplitude 150.0 uA
NRV INFO: ... Iteration simulation performed in 8.675652265548706 s
NRV INFO: ... Spike blocked
NRV INFO: Iteration number 5, testing block current amplitude 75.0 uA
NRV INFO: ... Iteration simulation performed in 9.020358085632324 s
NRV INFO: ... Spike blocked
NRV INFO: Iteration number 6, testing block current amplitude 37.5 uA
NRV INFO: 

IndexError: index 1 is out of bounds for axis 0 with size 1